<a href="https://colab.research.google.com/github/MohammadrezaPourreza/Keyword_generator/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers rouge-score nltk
!rm -rf t5-small-finetuned/

     |████████████████████████████████| 306 kB 7.7 MB/s 
     |████████████████████████████████| 3.4 MB 56.3 MB/s 
     |████████████████████████████████| 61 kB 511 kB/s 
     |████████████████████████████████| 1.1 MB 37.5 MB/s 
     |████████████████████████████████| 132 kB 51.8 MB/s 
     |████████████████████████████████| 243 kB 52.0 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.3 MB/s 
     |████████████████████████████████| 192 kB 61.9 MB/s 
     |████████████████████████████████| 160 kB 52.4 MB/s 
     |████████████████████████████████| 271 kB 51.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Login to HuggingFace**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,656 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


**Make sure our version is atleast 4.11.0**

In [ ]:
import transformers

print(transformers.__version__)

4.15.0


**Set the model checkpoint**

In [ ]:
model_checkpoint = "/content/drive/MyDrive/checkpoint-1000"

# **Loading the dataset**

**First import data from drive**

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
drive._mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Query_Abstract.csv',index_col = False)
df = df.sample(frac=1)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(['pubmed_id','keywords'], axis=1)
df = df.sample(frac=1).reset_index(drop=True)
df = df.head(100000)
test_df,val_df,train_df = np.split(df, [int(.08*len(df)), int(.16*len(df))])
print('length of the test dataset: ' + str(len(test_df.index)))
print('length of the validation dataset: ' + str(len(val_df.index)))
print('length of the train dataset: ' + str(len(train_df.index)))

Mounted at /content/drive
length of the test dataset: 8000
length of the validation dataset: 8000
length of the train dataset: 84000


In [ ]:
from datasets import load_dataset, load_metric, Dataset

metric = load_metric("rouge")
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)
del train_df
del val_df
del test_df

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

**Sanity check**

In [ ]:
test_data[1]

{'abstract': 'The flow of erythrocytes in parafoveal capillaries was imaged in the living human eye with an adaptive optics near-confocal ophthalmoscope at a frame rate of 800 Hz with a low coherence near-infrared (NIR) light source. Spatiotemporal traces of the erythrocyte movement were extracted from consecutive images. Erythrocyte velocity was measured using custom software based on the Radon transform. The impact of imaging speed on velocity measurement was estimated using images of frame rates of 200, 400, and 800 Hz. The NIR light allowed for long imaging periods without visually stimulating the retina and disturbing the natural rheological state. High speed near-confocal imaging enabled direct and accurate measurement of erythrocyte velocity, and revealed a distinctively cardiac-dependent pulsatile velocity waveform of the erythrocyte flow in retinal capillaries, disclosed the impact of the leukocytes on erythrocyte motion, and provided new metrics for precise assessment of eryt

**Look at the data**

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    print(df.to_markdown()) 

In [ ]:
show_random_elements(train_data)

|    | query                                                                                                                                | abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

**Check metric**

In [ ]:
fake_preds = ["hello Mohammadreza"]
fake_labels = ["hello Reza"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=0.5, recall=0.5, fmeasure=0.5), mid=Score(precision=0.5, recall=0.5, fmeasure=0.5), high=Score(precision=0.5, recall=0.5, fmeasure=0.5)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeL': AggregateScore(low=Score(precision=0.5, recall=0.5, fmeasure=0.5), mid=Score(precision=0.5, recall=0.5, fmeasure=0.5), high=Score(precision=0.5, recall=0.5, fmeasure=0.5)),
 'rougeLsum': AggregateScore(low=Score(precision=0.5, recall=0.5, fmeasure=0.5), mid=Score(precision=0.5, recall=0.5, fmeasure=0.5), high=Score(precision=0.5, recall=0.5, fmeasure=0.5))}

**Set the tokenizer**

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

**check**

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

**Processing the data**

In [ ]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["abstract"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["query"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

**Apply the function to the dataset**

In [ ]:
tokenized_datasets_train = train_data.map(preprocess_function, batched=True)
tokenized_datasets_val = val_data.map(preprocess_function, batched=True)
#tokenized_datasets_test = test_data.map(preprocess_function, batched=True)

  0%|          | 0/84 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

**Fine-Tuning the model**

In [ ]:
from transformers import EncoderDecoderModel, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 72
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned", #directory to save the model
    evaluation_strategy = "epoch",#evaluation after each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,#three saves maximum
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

**special data collector to pad the data**

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

**Evaluation method**

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Create the trainer**

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/MrezaPRZ/checkpoint-1000-finetuned into local empty directory.
Using amp half precision backend


copy the trained checkpoint

In [ ]:
%cp -r /content/checkpoint-1000-finetuned/checkpoint-1000 /content/drive/MyDrive

**Start Fine-tuning**

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: query, abstract.
***** Running training *****
  Num examples = 84000
  Num Epochs = 2
  Instantaneous batch size per device = 72
  Total train batch size (w. parallel, distributed & accumulation) = 72
  Gradient Accumulation steps = 1
  Total optimization steps = 2334


Epoch,Training Loss,Validation Loss


Saving model checkpoint to checkpoint-1000-finetuned/checkpoint-500
Configuration saved in checkpoint-1000-finetuned/checkpoint-500/config.json
Model weights saved in checkpoint-1000-finetuned/checkpoint-500/pytorch_model.bin
tokenizer config file saved in checkpoint-1000-finetuned/checkpoint-500/tokenizer_config.json
Special tokens file saved in checkpoint-1000-finetuned/checkpoint-500/special_tokens_map.json
tokenizer config file saved in checkpoint-1000-finetuned/tokenizer_config.json
Special tokens file saved in checkpoint-1000-finetuned/special_tokens_map.json


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.044300,2.817611,19.199600,1.878300,18.262200,18.271100,17.951600


Saving model checkpoint to checkpoint-1000-finetuned/checkpoint-1000
Configuration saved in checkpoint-1000-finetuned/checkpoint-1000/config.json
Model weights saved in checkpoint-1000-finetuned/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in checkpoint-1000-finetuned/checkpoint-1000/tokenizer_config.json
Special tokens file saved in checkpoint-1000-finetuned/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: query, abstract.
***** Running Evaluation *****
  Num examples = 8000
  Batch size = 72
Saving model checkpoint to checkpoint-1000-finetuned/checkpoint-1500
Configuration saved in checkpoint-1000-finetuned/checkpoint-1500/config.json
Model weights saved in checkpoint-1000-finetuned/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in checkpoint-1000-finetuned/checkpoint-1500/tokenizer_config.json
Special tokens file saved

**Evaluation**

In [ ]:
from transformers import EncoderDecoderModel,T5ForConditionalGeneration, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
from datasets import load_dataset, load_metric, Dataset
df = pd.read_csv('/content/drive/MyDrive/Query_Abstract_test.csv',index_col = False)
test_data = Dataset.from_pandas(df)

In [ ]:
#results = test_data.map(generate_query, batched=True, remove_columns=["abstract"])
for i in range(100,200,10):
  input_ids = tokenizer(test_data[i]['abstract'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids)
  #test_data["predicted"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print("predicted : " + tokenizer.decode(outputs[0], skip_special_tokens=True))
  print("original : "+test_data[i]['query'])
#test_data.to_csv("model_output.csv")

predicted : ( sulfate OR sulfate OR sulfate OR
original : ( CO(2) future estimations OR CO(2) sink OR CO(2) redistribution OR CO(2) source ) AND ( Underground cave OR Soil )
predicted : ( IL-2RA OR IL-2RA ) AND ( IL-33 OR
original : ( inflammation OR oxidative stress OR migraine OR mitochondria ) AND ( riboflavin )
predicted : ( GENOTOXYLE )
original : ( Drug–target interaction OR Read-across ) AND ( Gradient boosting OR Applicability Domain OR Prediction interval OR QSAR )
predicted : ( cytotoxic reactive oxygen species OR cytotoxic reactive oxygen species ) AND ( 
original : ( Uterine leiomyosarcoma OR Ovarian cancer OR Endometrial cancer OR Female cancer OR Cervical cancer ) AND ( Hedgehog pathway OR Therapeutic option )
predicted : ( ovulatory dysfunction OR ovulatory dysfunction ) AND ( ovul
original : ( meta-regression OR logistic regression OR stunting OR Forest plots )
predicted : ( ENTRY )
original : ( vaccine OR varicella ) AND ( ventilation and disinfection OR SEIR model OR 